# 2 Exercise 2

## 2.1 - Description

This code compares the the accuracy between a parametric optimal quantity estimation and a nonparametric optimal quantity estimation. To calculate the accuracy of the estimation, we use calculate the RSME of the optimal quantity and the PLR of the profit. All these values are exported to a ```.npy``` file, because this code is quite slow and then we can choose the graphs we want to create later. Note that this code was tested only on [Python 3.8](https://www.python.org/downloads/release/python-3810/). Altough Python 3.7 - 3.9 should work.

## 2.2 - Import dependencies

Import all the dependencies.
- ```datetime``` to add the current date to the filename of the ```.npy``` file.
- ```numpy``` for mathematical equations
- ```scipy.stats``` for statistical equations
- ```scipy.integrate``` for integrating
- ```matplotlib.pyplot``` for creating graphs
- ```typing``` for [type annotations](https://docs.python.org/3.8/library/typing.html)

In [1]:
import datetime
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

from scipy.integrate import quad, romberg
from typing import Tuple, List

## 2.3 - Auxiliary Functions

### 2.3.1 - Optimal percentage

**Purpose**: Function to calculate optimal percentage. Note that $F^{-1}_{Y}(\frac{\tilde{p}}{\tilde{p} + \tilde{c}}) = F^{-1}_Y(\frac{p - c}{p + c_h})$.<br/>
**Input**: ```price``` ($p$), ```cost``` ($c$), ```holding_cost``` ($c_h$).<br/>
**Output**: Optimal fraction.

In [2]:
def optimal_percentage(price: float, cost: float, holding_cost: float) -> float:
    return (price - cost)/(price + holding_cost)

### 2.3.2 - Calculate known optimal quantity (normal/lognormal)
**Purpose**: Calculate the *known* optimal quantity<br>
**Input**: ```cost``` ($c$), ```price``` ($p$), ```holding_cost``` ($c_h$), ```location```/```mu``` ($\mu$), ```scale```/```sigma``` ($\sigma$)<br>
**Output**: Optimal quantity ($Q^*$)

In [3]:
def calculate_known_optimal_quantity_normal(cost: float, price: float, holding_cost: float, location: float, scale: float) -> float:
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    return sts.norm.ppf(optimal_percent, loc=location, scale=scale)

def calculate_known_optimal_quantity_lognormal(cost: float, price: float, holding_cost: float, mu: float, sigma: float) -> float:
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    return sts.lognorm.ppf(optimal_percent, sigma, scale=np.exp(mu))

### 2.3.3 - Root Mean Squared Error (RSME)

**Purpose**: Calculate the Root Mean Squared Error $\textrm{RMSE}^k_n(\tau) = \sqrt{\frac{1}{M} \sum_{j=1}^{M}[{Q}^{k,j}_n(\tau) - Q^*(\tau)]^2}$, with $k \in \{\textrm{NP}, \textrm{P}\}$.<br>
**Input**: ```quantity_estimation_vector``` ($\pmb{\hat{Q}}$), ```known_optimal_quantity``` ($Q^*$). <br>
**Output**: $\textrm{RSME}_n^k(\tau)$, with $k \in \{\textrm{NP}, \textrm{P}\}$.

In [4]:
def root_mean_squared_error(quantity_estimation_vector: np.ndarray, known_optimal_quantity: float) -> float:
    estimation_error: np.ndarray = quantity_estimation_vector - known_optimal_quantity
    squared_error: float = np.square(estimation_error).sum()

    vector_size: int = quantity_estimation_vector.size
    mean_squared_error: float = squared_error/vector_size
    return np.sqrt(mean_squared_error)


### 2.3.4 - Emperical Profit Loss Ratio (PLR)

**Purpose**: Function that calculates the profit loss ratio. $\textrm{PLR}^k_n(\tau) = \frac{1}{M} \sum^M_{j=1} |\frac{R(Q^*; \tau) - R(\hat{Q}^{k,j}_n; \tau)}{R(Q^*; \tau)}|$.<br>
**Input**: ```profit_estimation_vector``` $(R(\hat{Q}^*_1; \tau), \ R(\hat{Q}^*_2; \tau), \ R(\hat{Q}^*_3; \tau), \ ...)$, ```known_maximum_profit``` $R(Q^*; \tau)$. <br>
**Output**: $\textrm{PLT}^k_n(\tau)$, with $k \in \{\textrm{NP}, \textrm{P}\}$.

In [5]:
def profit_loss_ratio(profit_estimation_vecor: np.ndarray, known_maximum_profit: float) -> float:
    ratio: np.ndarray = (known_maximum_profit - profit_estimation_vecor)/known_maximum_profit
    vector_size: int = profit_estimation_vecor.size
    return np.absolute(ratio).sum()/vector_size


## 2.4 Estimation Functions

### 2.4.1 Parametric Estimation (normal)

**Purpose**: Function that estimates the parameters from a normal/lognormal distribution.<br>
**Input**: ```data_vector``` ($\pmb{x}$)<br>
**Output**: ```mean``` ($\bar{x}$), ```variance``` $\textrm{Var}(x)$

### 2.4.2 Parametric Optimal Quantity (normal)

**Purpose**: Function that estimates the optimal quantity using the estimated location and scale.<br>
**Input**: ```cost``` ($c$), ```holding_cost``` ($c_h$), ```price``` ($p$), ```data_vector``` ($\pmb{x}$)<br>
**Output**: ```optimal_quantity``` ($Q^*$)

In [6]:
def normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]: # 2.4.1
    mean: float = data_vector.mean()
    var: float = data_vector.var()
    return (mean, np.sqrt(var))

def parametric_normal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float: # 2.4.2
    mean, std = normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float =  sts.norm.ppf(optimal_percent, loc=mean, scale=std)

    return inv_cdf_quantity

### 2.4.3 Parametric Estimation (lognormal)

**Purpose**: Function that estimates the parameters from a normal/lognormal distribution.<br>
**Input**: ```data_vector``` ($\pmb{x}$)<br>
**Output**: ```mu``` ($\overline{\ln{x}}$), ```variance``` $\textrm{Var}(\ln{x})$

### 2.4.4 Parametric Optimal Quantity (lognormal)

**Purpose**: Function that estimates the optimal quantity using the estimated location and scale.<br>
**Input**: ```cost``` ($c$), ```holding_cost``` ($c_h$), ```price``` ($p$), ```data_vector``` ($\pmb{x}$)<br>
**Output**: ```optimal_quantity``` ($Q^*$)

In [7]:
def log_normal_parametric_estimation(data_vector: np.ndarray) -> Tuple[float, float]: # 2.4.3
    mean: float = np.mean(np.log(data_vector))
    var: float = np.var(np.log(data_vector))
    return (mean, np.sqrt(var))

def parametric_lognormal_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float: # 2.4.4
    mean, std = log_normal_parametric_estimation(data_vector)
    optimal_percent: float = optimal_percentage(price, cost, holding_cost)
    inv_cdf_quantity: float = sts.lognorm.ppf(optimal_percent, std, scale=np.exp(mean))

    return inv_cdf_quantity

### 2.4.5 - Nonparametric Optimal Quantity
**Purpose**: Function that calculates the non parametric optimal quantity.<br>
**Input**: ```cost``` ($c$), ```holding_cost``` ($c_h$), ```price``` ($p$), ```data_vector``` ($\pmb{x}$) <br>
**Output**: ```optimal_quantity``` ($Q^*$)

In [8]:
def non_parametric_optimal_quantity(cost: float, holding_cost: float, price: float, data_vector: np.ndarray) -> float:
    data_vector.sort()
    vector_size = data_vector.size
    optimal_percent = optimal_percentage(price, cost, holding_cost)
    optimal_arg_value = int(np.ceil(optimal_percent * vector_size))

    return data_vector[max(optimal_arg_value - 1, 0)]

## 2.5 - Monte Carlo Simulations

### 2.5.1 Normal Simulation (parametric)

**Purpose**: Simulate data from a normal distribution, then return the estimated optimal quantity.<br>
**Input**: ```number``` ($n$, i.e. number of data points), ```cost``` ($c$), ```location``` ($\mu$), ```scale``` ($\sigma$), ```price``` ($p$)<br>
**Output**: Estimated optimal quantity ($\hat{Q}^*$)

In [9]:
def normal_monte_carlo_simulation_and_paramatric_estimation(number: float, cost: float, location: float, scale: float, price: float) -> float:
    data_vector: np.ndarray = np.random.normal(loc=location, scale=scale, size=number)
    return parametric_normal_optimal_quantity(cost, 0, price, data_vector)


### 2.5.2 Lognormal Simulation (parametric)

**Purpose**: Simulate data from a normal distribution, then return the estimated optimal quantity.<br>
**Input**: ```number``` ($n$, i.e. number of data points), ```cost``` ($c$), ```mu``` ($\mu$), ```sigma``` ($\sigma$), ```price``` ($p$)<br>
**Output**: Estimated optimal quantity ($\hat{Q}^*$)

In [10]:
def lognormal_monte_carlo_simulation_and_paramatric_estimation(number: float, cost: float, mu: float, sigma: float, price: float) -> float:
    data_vector: np.ndarray = np.random.lognormal(mu, sigma, size=number)
    return parametric_lognormal_optimal_quantity(cost, 0, price, data_vector)

### 2.5.3/2.5.4 (Log)Normal Simulation (nonparametric)

**Purpose**: Simulate data from a normal distribution, then return the estimated optimal quantity.<br>
**Input**: ```number``` ($n$, i.e. number of data points), ```cost``` ($c$), ```location``` ($\mu$), ```scale``` ($\sigma$), ```price``` ($p$).<br>
**Output**: Estimated optimal quantity ($\hat{Q}^*$).

In [11]:
def normal_monte_carlo_simulation_and_non_paramatric_estimation(number: float, cost: float, location: float, scale: float, price: float) -> float:
    data_vector: np.ndarray = np.random.normal(loc=location, scale=scale, size=number)
    return non_parametric_optimal_quantity(cost, 0, price, data_vector)

def lognormal_monte_carlo_simulation_and_non_paramatric_estimation(number: float, cost: float, mu: float, sigma: float, price: float) -> float:
    data_vector: np.ndarray = np.random.lognormal(mu, sigma, size=number)
    return non_parametric_optimal_quantity(cost, 0, price, data_vector)

## 2.6 Profit

### 2.6.1/2.6.2
**Purpose**: Calculate the expected profit from a normal or lognormal distributed function.<br>
**Input**: ```location_known``` ($\mu$), ```scale_known``` ($\sigma$), ```price``` ($p$), cost ($c$), ```quantity``` ($\hat{Q}^*$)<br>
**Output**: Expected profit $R(\hat{Q}^{k,j}_n; \ \tau)$


In [12]:
def calculate_normal_expected_profit(location_known: float, scale_known: float, price: float, cost: float, quantity: float) -> float:
    integral = quad(lambda y: sts.norm.cdf(y, loc=location_known, scale=scale_known), -np.inf, quantity)[0]
    return (price - cost) * quantity - price * integral

def calculate_lognormal_expected_profit(location_known: float, std_known: float, price: float, cost: float, quantity: float) -> float:
    integral = quad(lambda y: sts.lognorm.cdf(y, std_known, loc=np.exp(location_known)), -np.inf, quantity)[0]
    return (price - cost) * quantity - price * integral


## 2.7 Calculate RSME & PLR (parametric and nonparametric)

**Purpose**: Calculate the RSME and PLR<br>
**Input**: ```number```, ```target_surface```, ```scale```, ```price```, ```estimations```, ```location```<br>
**Output**: RSME, PLR

In [13]:
def calculate_normal_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float) -> Tuple[float, float]:
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)

    for i in range(estimations):
        quantity_estimation_results[i] = normal_monte_carlo_simulation_and_paramatric_estimation(number, 1 - target_surface, location, scale, price)
        profit_estimation_results[i] = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])

    known_optimal_quantity: float = calculate_known_optimal_quantity_normal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit)
    return (rsme, plr)
    

In [14]:
def calculate_lognormal_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float) -> Tuple[float, float]:
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)

    for i in range(estimations):
        quantity_estimation_results[i] = lognormal_monte_carlo_simulation_and_paramatric_estimation(number, target_surface, location, scale, price)
        #print(f"quantity: {quantity_estimation_results[i]}")
        profit_estimation_results[i] = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])
        #print(f"profit: {profit_estimation_results[i]}")

    known_optimal_quantity: float = calculate_known_optimal_quantity_lognormal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit)
    return (rsme, plr)

In [15]:
def calculate_normal_non_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float):
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)


    for i in range(estimations):
        quantity_estimation_results[i] = normal_monte_carlo_simulation_and_non_paramatric_estimation(number, target_surface, location, scale, price)
        profit_estimation_results[i] = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])

    known_optimal_quantity: float = calculate_known_optimal_quantity_normal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_normal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit) # Calculate
    return (rsme, plr)

def calculate_lognormal_non_parametric_metrics(number: int, target_surface: float, scale: float, price: float, estimations: int, location: float):
    quantity_estimation_results: np.ndarray = np.empty(estimations)
    profit_estimation_results: np.ndarray = np.empty(estimations)


    for i in range(estimations):
        quantity_estimation_results[i] = lognormal_monte_carlo_simulation_and_non_paramatric_estimation(number, target_surface, location, scale, price)
        profit_estimation_results[i] = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, quantity_estimation_results[i])

    known_optimal_quantity: float = calculate_known_optimal_quantity_lognormal(1 - target_surface, price, 0, location, scale)
    known_maximum_profit: float = calculate_lognormal_expected_profit(location, scale, price, 1 - target_surface, known_optimal_quantity)
    rsme: float = root_mean_squared_error(quantity_estimation_results, known_optimal_quantity)
    plr: float = profit_loss_ratio(profit_estimation_results, known_maximum_profit) # Calculate
    return (rsme, plr)

## 2.8 - Total calculations

In [16]:
def normal():
    date: str = datetime.datetime.utcnow().strftime("%y%m%d-%H%M%S")
    locations: List[int] = [50, 100, 200, 500]
    scales: List[int] = [1, 5, 10, 20]
    estimations: int = 1000
    number: List[int] = [10, 50, 100, 200]
    target_surface: List[int] = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
    price: int = 1

    output_data = np.zeros((len(locations), len(scales), len(number), len(target_surface), 2, 2))

    for a, location in enumerate(locations):
        for b, scale in enumerate(scales):
            for c, n in enumerate(number):
                for d, t in enumerate(target_surface):
                    print(f"n: {n}, t: {t}, loc: {location}, scale: {scale}")
                    parametric_metrics = calculate_normal_parametric_metrics(n, t, scale, price, estimations, location)
                    nonparametric_metrics = calculate_normal_non_parametric_metrics(n, t, scale, price, estimations, location)
                    print(f"parametric rsme: {parametric_metrics[0]}, plr: {parametric_metrics[1]}")
                    print(f"nonparametric rsme: {nonparametric_metrics[0]}, plr {nonparametric_metrics[1]}")
                    print("")

                    output_data[a][b][c][d] = np.array([parametric_metrics, nonparametric_metrics])

                    with open(f"r-norm-{date}.npy", "wb") as file:
                        np.save(file, output_data) # Save on every run because process might crash

In [ ]:
print("start")
normal()
print("finished")

In [18]:
def lognormal():
    date: str = datetime.datetime.utcnow().strftime("%y%m%d-%H%M%S")
    locations: List[int] = [4, 5, 6, 7, 8]#[50, 100, 200, 500]
    scales: List[int] = [0.2, 0.5, 0.8, 1] #[1, 5, 10, 20]
    estimations: int = 1000
    number: List[int] = [10, 50, 100, 200]
    target_surface: List[int] = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
    price: int = 1

    output_data = np.zeros((len(locations), len(scales), len(number), len(target_surface), 2, 2))

    for a, location in enumerate(locations):
        for b, scale in enumerate(scales):
            for c, n in enumerate(number):
                for d, t in enumerate(target_surface):
                    s = datetime.datetime.now()
                    print(f"n: {n}, t: {t}, loc: {location}, scale: {scale}")
                    parametric_metrics = calculate_lognormal_parametric_metrics(n, t, scale, price, estimations, location)
                    nonparametric_metrics = calculate_lognormal_non_parametric_metrics(n, t, scale, price, estimations, location)
                    print(f"parametric rsme: {parametric_metrics[0]}, plr: {parametric_metrics[1]}")
                    print(f"nonparametric rsme: {nonparametric_metrics[0]}, plr {nonparametric_metrics[1]}")
                    print(datetime.datetime.now() - s)
                    print("")

                    output_data[a][b][c][d] = np.array([parametric_metrics, nonparametric_metrics])

                    with open(f"r-lognorm-{date}.npy", "wb") as file:
                        np.save(file, output_data) # Save on every run because process might crash

In [ ]:
print("start")
lognormal()
print("finished")

# Excercise 3

In [20]:
def lognormal_monte_carlo_simulation_and_normal_paramatric_estimation(number: float, cost: float, mu: float, sigma: float, price: float) -> float:
    data_vector: np.ndarray = np.random.lognormal(mu, sigma, size=number)
    return parametric_normal_optimal_quantity(cost, 0, price, data_vector)

In [44]:
def lognormal_as_normal():
    cost = 0.5
    price = 1
    mu = 5
    sigma = 0.5
    estimations = 10
    number = [10, 50, 100, 200]

    for n in number:
        quantity_estimations = np.zeros(estimations)
        profit_of_estimations = np.zeros(estimations)
        for i in range(estimations):
            quantity_estimations[i] = lognormal_monte_carlo_simulation_and_normal_paramatric_estimation(number, cost, mu, sigma, price)
            profit_of_estimations[i] = calculate_lognormal_expected_profit(mu, sigma, price, cost, quantity_estimations[i])

        known_optimal_quantity = calculate_known_optimal_quantity_lognormal(cost, price, 0, mu, sigma)
        known_maximum_profit = calculate_lognormal_expected_profit(mu, sigma, price, cost, known_optimal_quantity)


        rsme = root_mean_squared_error(quantity_estimations, known_optimal_quantity)
        plr = profit_loss_ratio(profit_of_estimations, known_maximum_profit)

        print(f"n: {n}")
        print(f"rsme: {rsme}, plr: {plr}")
        print("")

In [45]:
print("start")
lognormal_as_normal()
print("finished")

start
n: 10
rsme: 19.772958370837475, plr: 0.11795888182243433
n: 50
rsme: 19.776163905898198, plr: 0.11798033562797547
n: 100
rsme: 19.767176296684937, plr: 0.11791984048533961
n: 200
rsme: 19.756021481555678, plr: 0.11784474240942258
finished
